In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

gpg: keybox '/tmp/tmpjye8btw_/pubring.gpg' created
gpg: /tmp/tmpjye8btw_/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
folder='./drive/ML/Cifar-10/cifar-10-batches-py'

In [0]:
import pandas as pd
import numpy as np
import os
import pickle


In [0]:
def unpickle(file):
  
  '''Load byte data from file'''
  with open(file, 'rb') as f:
    data = pickle.load(f)
    return data


def load_cifar10_data(data_dir):
  '''Return train_data, train_labels, test_data, test_labels
   The shape of data is 32 x 32 x3'''
    
  train_data = None
  train_labels = []

  for i in range(1, 6):
    data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
    if i == 1:
      train_data = data_dic['data']
    else:
      train_data = np.vstack((train_data, data_dic['data']))
    train_labels += data_dic['labels']
  test_data_dic = unpickle(data_dir + "/test_batch")
  test_data = test_data_dic['data']
  test_labels = test_data_dic['labels']

  train_data = train_data.reshape((len(train_data), 3, 32, 32))
  train_data = np.rollaxis(train_data, 1, 4)
  train_labels = np.array(train_labels)

  test_data = test_data.reshape((len(test_data), 3, 32, 32))
  test_data = np.rollaxis(test_data, 1, 4)
  test_labels = np.array(test_labels)

  return train_data, train_labels, test_data, test_labels

In [0]:
train_data, train_labels, test_data, test_labels = load_cifar10_data(folder)


In [7]:
print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)


(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [8]:
import keras

Using TensorFlow backend.


In [0]:
y_train=np.asarray(train_labels).reshape(-1,1)
y_train=keras.utils.to_categorical(y_train,num_classes=10)

y_test=np.asarray(test_labels).reshape(-1,1)
y_test=keras.utils.to_categorical(y_test,num_classes=10)



In [0]:
x_train=train_data.astype('float')
x_test=test_data.astype('float')
x_train/=255.0
x_test/=255.0

In [0]:
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Input,AveragePooling2D,BatchNormalization,Activation,GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy,categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator

In [12]:
datagen=ImageDataGenerator(featurewise_std_normalization=True,horizontal_flip=True)
datagen.fit(x_train)
train_gen=datagen.flow(x_train,y_train,batch_size=256,shuffle=True)
test_gen=datagen.flow(x_test,y_test,batch_size=256,shuffle=True)

/usr/local/lib/python2.7/dist-packages/keras/preprocessing/image.py:657: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
def residual_layer(y,channels,_strides=(1,1),project_shortcut=True):
    shortcut=y
    y=Conv2D(channels,kernel_size=(3,3),strides=_strides,activation='relu',padding='same')(y)
    y=BatchNormalization()(y)
    y=Activation('relu')(y)
    
    y=Conv2D(channels,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(y)
    y=BatchNormalization()(y)
    y=Activation('relu')(y)
    
    if(_strides!=(1,1)):
        shortcut=Conv2D(channels,kernel_size=(1,1),strides=_strides,activation='relu',padding='same')(shortcut)
        shortcut=BatchNormalization()(shortcut)
        
    y=keras.layers.add([shortcut,y])
    y=Activation('relu')(y)
    return y
    
    
    
    
    

In [0]:
def make_model(classes=10):
    inputs=Input(shape=(32,32,3))
    l1=Conv2D(16,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(inputs)
    l1=BatchNormalization()(l1)
    l1=Activation('relu')(l1)
    
    
    
    l2=residual_layer(l1,16)
    l3=residual_layer(l2,16)
    l4=residual_layer(l3,16)
    l5=residual_layer(l4,16)
    l6=residual_layer(l5,16)
    
    l7=residual_layer(l6,32,(2,2))
    l8=residual_layer(l7,32)
    l9=residual_layer(l8,32)
    l10=residual_layer(l9,32)
    l11=residual_layer(l10,32)
    
    
    l12=residual_layer(l11,64,(2,2))
    l13=residual_layer(l12,64)
    l14=residual_layer(l13,64)
    l15=residual_layer(l14,64)
    l16=residual_layer(l15,64)
    
    l17=GlobalAveragePooling2D()(l16)
    outputs=Dense(classes,activation='softmax')(l17)
    
    return inputs,outputs
    
    

In [15]:
inputs,outputs=make_model()
resnet34=Model(inputs=inputs,outputs=outputs)
resnet34.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
optim=SGD(1e-1,momentum=0.9,decay=0.0001)
loss=categorical_crossentropy
resnet34.compile(optim,loss,metrics=['accuracy'])

In [25]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=20,verbose=1)

Epoch 1/20
195/195 [==============================] - 69s 356ms/step - loss: 1.2291 - acc: 0.5933 - val_loss: 4.4687 - val_acc: 0.3277
Epoch 2/20
118/195 [=================>............] - ETA: 23s - loss: 0.7898 - acc: 0.7261

195/195 [==============================] - 64s 330ms/step - loss: 0.7595 - acc: 0.7361 - val_loss: 0.8897 - val_acc: 0.6887
Epoch 3/20
195/195 [==============================] - 64s 328ms/step - loss: 0.6158 - acc: 0.7872 - val_loss: 0.9055 - val_acc: 0.7045
Epoch 4/20
146/195 [=====================>........] - ETA: 15s - loss: 0.5360 - acc: 0.8146

195/195 [==============================] - 64s 328ms/step - loss: 0.5317 - acc: 0.8154 - val_loss: 0.7356 - val_acc: 0.7560
Epoch 5/20
195/195 [==============================] - 64s 329ms/step - loss: 0.4725 - acc: 0.8359 - val_loss: 0.9485 - val_acc: 0.7304
Epoch 6/20
153/195 [======================>.......] - ETA: 12s - loss: 0.4180 - acc: 0.8550

195/195 [==============================] - 64s 328ms/step - loss: 0.4203 - acc: 0.8540 - val_loss: 0.6635 - val_acc: 0.7860
Epoch 7/20
195/195 [==============================] - 64s 328ms/step - loss: 0.3902 - acc: 0.8628 - val_loss: 0.6129 - val_acc: 0.8012
Epoch 8/20
154/195 [======================>.......] - ETA: 12s - loss: 0.3471 - acc: 0.8796

195/195 [==============================] - 64s 329ms/step - loss: 0.3530 - acc: 0.8778 - val_loss: 0.6875 - val_acc: 0.7809
Epoch 9/20
195/195 [==============================] - 65s 333ms/step - loss: 0.3239 - acc: 0.8879 - val_loss: 0.7704 - val_acc: 0.7645
Epoch 10/20
154/195 [======================>.......] - ETA: 12s - loss: 0.2940 - acc: 0.8966

...

195/195 [==============================] - 64s 330ms/step - loss: 0.2611 - acc: 0.9097 - val_loss: 0.7885 - val_acc: 0.7777
Epoch 13/20
195/195 [==============================] - 65s 332ms/step - loss: 0.2313 - acc: 0.9199 - val_loss: 0.6786 - val_acc: 0.7987
Epoch 14/20
153/195 [======================>.......] - ETA: 13s - loss: 0.2022 - acc: 0.9297

195/195 [==============================] - 65s 332ms/step - loss: 0.2070 - acc: 0.9279 - val_loss: 0.8863 - val_acc: 0.7554
Epoch 15/20
195/195 [==============================] - 65s 332ms/step - loss: 0.1972 - acc: 0.9318 - val_loss: 0.6919 - val_acc: 0.8041
Epoch 16/20
153/195 [======================>.......] - ETA: 13s - loss: 0.1854 - acc: 0.9350

195/195 [==============================] - 65s 332ms/step - loss: 0.1889 - acc: 0.9338 - val_loss: 0.9445 - val_acc: 0.7652
Epoch 17/20
195/195 [==============================] - 64s 330ms/step - loss: 0.1695 - acc: 0.9416 - val_loss: 0.7440 - val_acc: 0.7991
Epoch 18/20
153/195 [======================>.......] - ETA: 13s - loss: 0.1559 - acc: 0.9447

195/195 [==============================] - 65s 332ms/step - loss: 0.1597 - acc: 0.9434 - val_loss: 0.6617 - val_acc: 0.8186
Epoch 19/20
195/195 [==============================] - 65s 332ms/step - loss: 0.1429 - acc: 0.9496 - val_loss: 0.7728 - val_acc: 0.7950
Epoch 20/20
153/195 [======================>.......] - ETA: 12s - loss: 0.1352 - acc: 0.9525

195/195 [==============================] - 65s 332ms/step - loss: 0.1370 - acc: 0.9518 - val_loss: 1.0492 - val_acc: 0.7717


In [0]:
resnet34.save_weights("./drive/ML/Cifar-10/weights_8572.h5")

In [27]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=15,verbose=1)

Epoch 1/15
195/195 [==============================] - 65s 333ms/step - loss: 0.1174 - acc: 0.9584 - val_loss: 0.7853 - val_acc: 0.8136
Epoch 2/15
195/195 [==============================] - 65s 332ms/step - loss: 0.1123 - acc: 0.9622 - val_loss: 0.7300 - val_acc: 0.8118
Epoch 3/15
104/195 [===============>..............] - ETA: 28s - loss: 0.0967 - acc: 0.9655

195/195 [==============================] - 65s 333ms/step - loss: 0.1023 - acc: 0.9637 - val_loss: 0.7773 - val_acc: 0.8196
Epoch 4/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0968 - acc: 0.9660 - val_loss: 0.6675 - val_acc: 0.8348
Epoch 5/15
143/195 [=====================>........] - ETA: 16s - loss: 0.0784 - acc: 0.9730

195/195 [==============================] - 65s 331ms/step - loss: 0.0820 - acc: 0.9714 - val_loss: 0.8307 - val_acc: 0.8138
Epoch 6/15
195/195 [==============================] - 65s 331ms/step - loss: 0.0763 - acc: 0.9744 - val_loss: 0.8083 - val_acc: 0.8079
Epoch 7/15
152/195 [======================>.......] - ETA: 13s - loss: 0.0648 - acc: 0.9772

195/195 [==============================] - 65s 333ms/step - loss: 0.0663 - acc: 0.9766 - val_loss: 0.8268 - val_acc: 0.8149
Epoch 8/15
195/195 [==============================] - 64s 331ms/step - loss: 0.0588 - acc: 0.9798 - val_loss: 0.8625 - val_acc: 0.8149
Epoch 9/15
154/195 [======================>.......] - ETA: 12s - loss: 0.0586 - acc: 0.9803

195/195 [==============================] - 65s 331ms/step - loss: 0.0591 - acc: 0.9798 - val_loss: 0.8714 - val_acc: 0.8194
Epoch 10/15
195/195 [==============================] - 64s 330ms/step - loss: 0.0546 - acc: 0.9812 - val_loss: 0.8814 - val_acc: 0.8168
Epoch 11/15
153/195 [======================>.......] - ETA: 12s - loss: 0.0514 - acc: 0.9823

195/195 [==============================] - 64s 329ms/step - loss: 0.0534 - acc: 0.9812 - val_loss: 0.8590 - val_acc: 0.8238
Epoch 12/15
195/195 [==============================] - 64s 329ms/step - loss: 0.0507 - acc: 0.9824 - val_loss: 0.7916 - val_acc: 0.8302
Epoch 13/15
153/195 [======================>.......] - ETA: 12s - loss: 0.0397 - acc: 0.9868

195/195 [==============================] - 64s 330ms/step - loss: 0.0411 - acc: 0.9863 - val_loss: 0.7971 - val_acc: 0.8369
Epoch 14/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0446 - acc: 0.9842 - val_loss: 0.7848 - val_acc: 0.8406
Epoch 15/15
153/195 [======================>.......] - ETA: 12s - loss: 0.0368 - acc: 0.9874

195/195 [==============================] - 65s 331ms/step - loss: 0.0369 - acc: 0.9875 - val_loss: 0.7849 - val_acc: 0.8382


In [29]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=15,verbose=1)

Epoch 1/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0248 - acc: 0.9920 - val_loss: 1.0256 - val_acc: 0.8142
Epoch 2/15
195/195 [==============================] - 65s 331ms/step - loss: 0.0217 - acc: 0.9933 - val_loss: 0.8001 - val_acc: 0.8452
Epoch 3/15
104/195 [===============>..............] - ETA: 28s - loss: 0.0180 - acc: 0.9938

195/195 [==============================] - 65s 331ms/step - loss: 0.0194 - acc: 0.9939 - val_loss: 1.0222 - val_acc: 0.8253
Epoch 4/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0177 - acc: 0.9945 - val_loss: 0.8324 - val_acc: 0.8415
Epoch 5/15
143/195 [=====================>........] - ETA: 16s - loss: 0.0168 - acc: 0.9947

195/195 [==============================] - 64s 329ms/step - loss: 0.0165 - acc: 0.9949 - val_loss: 0.8208 - val_acc: 0.8447
Epoch 6/15
195/195 [==============================] - 64s 330ms/step - loss: 0.0156 - acc: 0.9955 - val_loss: 0.9789 - val_acc: 0.8278
Epoch 7/15
152/195 [======================>.......] - ETA: 13s - loss: 0.0173 - acc: 0.9947

195/195 [==============================] - 65s 332ms/step - loss: 0.0173 - acc: 0.9945 - val_loss: 0.8970 - val_acc: 0.8393
Epoch 8/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0125 - acc: 0.9961 - val_loss: 0.8586 - val_acc: 0.8431
Epoch 9/15
154/195 [======================>.......] - ETA: 12s - loss: 0.0098 - acc: 0.9973

195/195 [==============================] - 65s 332ms/step - loss: 0.0094 - acc: 0.9974 - val_loss: 0.8837 - val_acc: 0.8451
Epoch 10/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0126 - acc: 0.9962 - val_loss: 0.9296 - val_acc: 0.8341
Epoch 11/15
153/195 [======================>.......] - ETA: 13s - loss: 0.0105 - acc: 0.9971

195/195 [==============================] - 65s 332ms/step - loss: 0.0101 - acc: 0.9972 - val_loss: 0.8733 - val_acc: 0.8446
Epoch 12/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0079 - acc: 0.9978 - val_loss: 0.9334 - val_acc: 0.8389
Epoch 13/15
153/195 [======================>.......] - ETA: 12s - loss: 0.0065 - acc: 0.9983

195/195 [==============================] - 65s 332ms/step - loss: 0.0065 - acc: 0.9983 - val_loss: 0.9006 - val_acc: 0.8446
Epoch 14/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0084 - acc: 0.9978 - val_loss: 0.8981 - val_acc: 0.8477
Epoch 15/15
153/195 [======================>.......] - ETA: 12s - loss: 0.0045 - acc: 0.9991

195/195 [==============================] - 65s 332ms/step - loss: 0.0050 - acc: 0.9989 - val_loss: 0.9123 - val_acc: 0.8458


In [31]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=15,verbose=1)

Epoch 1/15
195/195 [==============================] - 65s 333ms/step - loss: 0.0034 - acc: 0.9993 - val_loss: 0.8947 - val_acc: 0.8536
Epoch 2/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0031 - acc: 0.9994 - val_loss: 0.8973 - val_acc: 0.8494
Epoch 3/15
104/195 [===============>..............] - ETA: 28s - loss: 0.0023 - acc: 0.9996

195/195 [==============================] - 65s 333ms/step - loss: 0.0021 - acc: 0.9997 - val_loss: 0.8787 - val_acc: 0.8562
Epoch 4/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0018 - acc: 0.9997 - val_loss: 0.8945 - val_acc: 0.8534
Epoch 5/15
143/195 [=====================>........] - ETA: 15s - loss: 0.0019 - acc: 0.9996

195/195 [==============================] - 64s 329ms/step - loss: 0.0018 - acc: 0.9996 - val_loss: 0.9130 - val_acc: 0.8540
Epoch 6/15
195/195 [==============================] - 64s 329ms/step - loss: 9.7837e-04 - acc: 0.9999 - val_loss: 0.8760 - val_acc: 0.8565
Epoch 7/15
147/195 [=====================>........] - ETA: 14s - loss: 9.9030e-04 - acc: 0.9999

195/195 [==============================] - 65s 332ms/step - loss: 9.3300e-04 - acc: 0.9999 - val_loss: 0.8935 - val_acc: 0.8558
Epoch 8/15
195/195 [==============================] - 65s 332ms/step - loss: 8.3467e-04 - acc: 1.0000 - val_loss: 0.8982 - val_acc: 0.8557
Epoch 9/15
143/195 [=====================>........] - ETA: 15s - loss: 6.1183e-04 - acc: 1.0000

195/195 [==============================] - 64s 326ms/step - loss: 5.9154e-04 - acc: 1.0000 - val_loss: 0.8911 - val_acc: 0.8582
Epoch 10/15
195/195 [==============================] - 64s 327ms/step - loss: 5.3548e-04 - acc: 1.0000 - val_loss: 0.8803 - val_acc: 0.8561
Epoch 11/15
141/195 [====================>.........] - ETA: 16s - loss: 5.4815e-04 - acc: 1.0000

195/195 [==============================] - 64s 331ms/step - loss: 5.2250e-04 - acc: 1.0000 - val_loss: 0.9188 - val_acc: 0.8590
Epoch 12/15
195/195 [==============================] - 64s 329ms/step - loss: 4.8260e-04 - acc: 1.0000 - val_loss: 0.8889 - val_acc: 0.8568
Epoch 13/15
140/195 [====================>.........] - ETA: 17s - loss: 4.9449e-04 - acc: 1.0000

195/195 [==============================] - 65s 332ms/step - loss: 4.9542e-04 - acc: 1.0000 - val_loss: 0.8876 - val_acc: 0.8575
Epoch 14/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0012 - acc: 0.9998 - val_loss: 0.9170 - val_acc: 0.8526
Epoch 15/15
144/195 [=====================>........] - ETA: 15s - loss: 5.9299e-04 - acc: 1.0000

195/195 [==============================] - 65s 332ms/step - loss: 6.2380e-04 - acc: 1.0000 - val_loss: 0.9072 - val_acc: 0.8572


In [0]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=20,verbose=1)

Epoch 1/20
195/195 [==============================] - 65s 333ms/step - loss: 4.1787e-04 - acc: 1.0000 - val_loss: 0.9019 - val_acc: 0.8575
Epoch 2/20
195/195 [==============================] - 65s 333ms/step - loss: 4.0950e-04 - acc: 1.0000 - val_loss: 0.8976 - val_acc: 0.8626
Epoch 3/20
 94/195 [=============>................] - ETA: 31s - loss: 3.6737e-04 - acc: 1.0000

195/195 [==============================] - 65s 332ms/step - loss: 3.7145e-04 - acc: 1.0000 - val_loss: 0.9229 - val_acc: 0.8593
Epoch 4/20
195/195 [==============================] - 65s 332ms/step - loss: 6.1552e-04 - acc: 0.9999 - val_loss: 0.9344 - val_acc: 0.8575
Epoch 5/20
131/195 [===================>..........] - ETA: 19s - loss: 4.4524e-04 - acc: 1.0000

195/195 [==============================] - 65s 333ms/step - loss: 4.5950e-04 - acc: 1.0000 - val_loss: 0.9228 - val_acc: 0.8564
Epoch 6/20
195/195 [==============================] - 65s 332ms/step - loss: 4.8111e-04 - acc: 1.0000 - val_loss: 0.9260 - val_acc: 0.8538
Epoch 7/20
140/195 [====================>.........] - ETA: 17s - loss: 4.7697e-04 - acc: 1.0000

195/195 [==============================] - 65s 332ms/step - loss: 4.2727e-04 - acc: 1.0000 - val_loss: 0.9469 - val_acc: 0.8551
Epoch 8/20
195/195 [==============================] - 65s 332ms/step - loss: 3.6268e-04 - acc: 1.0000 - val_loss: 0.9178 - val_acc: 0.8545
Epoch 9/20
141/195 [====================>.........] - ETA: 16s - loss: 3.1579e-04 - acc: 1.0000

195/195 [==============================] - 64s 328ms/step - loss: 3.1671e-04 - acc: 1.0000 - val_loss: 0.9048 - val_acc: 0.8591
Epoch 10/20
195/195 [==============================] - 64s 327ms/step - loss: 2.7627e-04 - acc: 1.0000 - val_loss: 0.9461 - val_acc: 0.8533
Epoch 11/20
141/195 [====================>.........] - ETA: 16s - loss: 6.1395e-04 - acc: 0.9999

195/195 [==============================] - 64s 327ms/step - loss: 5.8010e-04 - acc: 0.9999 - val_loss: 0.9521 - val_acc: 0.8574
Epoch 12/20
195/195 [==============================] - 64s 330ms/step - loss: 3.2594e-04 - acc: 1.0000 - val_loss: 0.9423 - val_acc: 0.8585
Epoch 13/20
140/195 [====================>.........] - ETA: 17s - loss: 2.9423e-04 - acc: 1.0000

195/195 [==============================] - 65s 333ms/step - loss: 2.9134e-04 - acc: 1.0000 - val_loss: 0.9457 - val_acc: 0.8574
Epoch 14/20
195/195 [==============================] - 65s 331ms/step - loss: 2.4101e-04 - acc: 1.0000 - val_loss: 0.9392 - val_acc: 0.8567
Epoch 15/20
 68/195 [=========>....................] - ETA: 39s - loss: 2.3387e-04 - acc: 1.0000

In [34]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=20,verbose=1)

Epoch 1/20
195/195 [==============================] - 64s 330ms/step - loss: 1.6122e-04 - acc: 1.0000 - val_loss: 0.9262 - val_acc: 0.8583
Epoch 2/20
195/195 [==============================] - 64s 330ms/step - loss: 1.8567e-04 - acc: 1.0000 - val_loss: 0.9501 - val_acc: 0.8579
Epoch 3/20
 94/195 [=============>................] - ETA: 30s - loss: 2.4403e-04 - acc: 1.0000

195/195 [==============================] - 64s 326ms/step - loss: 2.2040e-04 - acc: 1.0000 - val_loss: 0.9374 - val_acc: 0.8567
Epoch 4/20
195/195 [==============================] - 64s 330ms/step - loss: 1.8332e-04 - acc: 1.0000 - val_loss: 0.9325 - val_acc: 0.8586
Epoch 5/20
131/195 [===================>..........] - ETA: 19s - loss: 1.7314e-04 - acc: 1.0000

195/195 [==============================] - 64s 329ms/step - loss: 1.7265e-04 - acc: 1.0000 - val_loss: 0.9439 - val_acc: 0.8567
Epoch 6/20
195/195 [==============================] - 64s 329ms/step - loss: 1.5886e-04 - acc: 1.0000 - val_loss: 0.9559 - val_acc: 0.8569
Epoch 7/20
139/195 [====================>.........] - ETA: 17s - loss: 2.4571e-04 - acc: 1.0000

195/195 [==============================] - 64s 330ms/step - loss: 2.2773e-04 - acc: 1.0000 - val_loss: 0.9375 - val_acc: 0.8590
Epoch 8/20
195/195 [==============================] - 64s 327ms/step - loss: 9.2803e-04 - acc: 0.9999 - val_loss: 0.9617 - val_acc: 0.8545
Epoch 9/20
143/195 [=====================>........] - ETA: 15s - loss: 3.5427e-04 - acc: 1.0000

195/195 [==============================] - 64s 329ms/step - loss: 3.2067e-04 - acc: 1.0000 - val_loss: 0.9478 - val_acc: 0.8571
Epoch 10/20
195/195 [==============================] - 64s 327ms/step - loss: 4.0010e-04 - acc: 0.9999 - val_loss: 1.0621 - val_acc: 0.8444
Epoch 11/20
141/195 [====================>.........] - ETA: 16s - loss: 3.2264e-04 - acc: 1.0000

195/195 [==============================] - 64s 327ms/step - loss: 3.1245e-04 - acc: 1.0000 - val_loss: 0.9734 - val_acc: 0.8576
Epoch 12/20
195/195 [==============================] - 65s 332ms/step - loss: 8.1266e-04 - acc: 0.9998 - val_loss: 0.9816 - val_acc: 0.8523
Epoch 13/20
141/195 [====================>.........] - ETA: 16s - loss: 4.0561e-04 - acc: 1.0000

195/195 [==============================] - 64s 329ms/step - loss: 3.9414e-04 - acc: 1.0000 - val_loss: 0.9575 - val_acc: 0.8560
Epoch 14/20
195/195 [==============================] - 65s 332ms/step - loss: 2.5020e-04 - acc: 1.0000 - val_loss: 0.9718 - val_acc: 0.8552
Epoch 15/20
140/195 [====================>.........] - ETA: 17s - loss: 2.2314e-04 - acc: 1.0000

195/195 [==============================] - 65s 333ms/step - loss: 2.1825e-04 - acc: 1.0000 - val_loss: 0.9700 - val_acc: 0.8570
Epoch 16/20
195/195 [==============================] - 64s 330ms/step - loss: 2.1104e-04 - acc: 1.0000 - val_loss: 0.9467 - val_acc: 0.8575
Epoch 17/20
140/195 [====================>.........] - ETA: 16s - loss: 1.8078e-04 - acc: 1.0000

195/195 [==============================] - 65s 331ms/step - loss: 1.9524e-04 - acc: 1.0000 - val_loss: 0.9487 - val_acc: 0.8575
Epoch 18/20
195/195 [==============================] - 65s 331ms/step - loss: 1.9473e-04 - acc: 1.0000 - val_loss: 0.9380 - val_acc: 0.8583
Epoch 19/20
140/195 [====================>.........] - ETA: 16s - loss: 1.6294e-04 - acc: 1.0000

195/195 [==============================] - 65s 331ms/step - loss: 1.7188e-04 - acc: 1.0000 - val_loss: 0.9611 - val_acc: 0.8578
Epoch 20/20
195/195 [==============================] - 65s 332ms/step - loss: 1.7350e-04 - acc: 1.0000 - val_loss: 0.9833 - val_acc: 0.8562
